In [30]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [31]:
s = Service('C:/Users/Athar/Downloads/chromedriver-win64 (1)/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=s)

In [32]:
url = 'https://clutch.co/it-services/analytics/power-bi?geona_id=840&page=1'
driver.get(url)

In [33]:
 driver.execute_script('window.scrollTo(0,16500)');

In [35]:
import time

while True:
    try:
        driver.execute_script('window.scrollTo(0, 16500)')
        time.sleep(1)  # Adding a short delay to allow time for elements to become clickable
        next_page = driver.find_element(By.XPATH, '/html/body/main/section[1]/nav/ul/li[9]/a')
        next_page.click()
    except Exception as e:
        print(f"An error occurred: {e}")
        break


An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.61)
Stacktrace:
	GetHandleVerifier [0x00007FF75DA022C2+60002]
	(No symbol) [0x00007FF75D97CA59]
	(No symbol) [0x00007FF75D837EDA]
	(No symbol) [0x00007FF75D80D5B5]
	(No symbol) [0x00007FF75D8B3727]
	(No symbol) [0x00007FF75D8CB3A1]
	(No symbol) [0x00007FF75D8AC033]
	(No symbol) [0x00007FF75D879657]
	(No symbol) [0x00007FF75D87A251]
	GetHandleVerifier [0x00007FF75DD13E2D+3278285]
	GetHandleVerifier [0x00007FF75DD60190+3590448]
	GetHandleVerifier [0x00007FF75DD561D0+3549552]
	GetHandleVerifier [0x00007FF75DAB1DE6+779654]
	(No symbol) [0x00007FF75D987ACF]
	(No symbol) [0x00007FF75D982EE4]
	(No symbol) [0x00007FF75D983072]
	(No symbol) [0x00007FF75D972C4F]
	BaseThreadInitThunk [0x00007FF8A50F7344+20]
	RtlUserThreadStart [0x00007FF8A70626B1+33]



In [17]:
names_list = []
links_list = []

# Maximum number of iterations
max_iterations = 100
iterations = 0

while True:
    try:
        soup = BeautifulSoup(driver.page_source, 'lxml')    
        time.sleep(3)
        driver.execute_script('window.scrollTo(0, 17000);')

        names = soup.find_all('div', class_='row provider-info--header')
        links = soup.find_all('a', class_='website-link__item')
        
        for name, link in zip(names, links):
            name_text = name.get_text(strip=True)
            link_href = link.get('href')
            if name_text and link_href:  # Check if both name and link are present
                names_list.append(name_text)
                links_list.append(link_href)

                try:
                    driver.execute_script('window.scrollTo(0, 16500)')
                    time.sleep(1)  # Adding a short delay to allow time for elements to become clickable
                    next_page = driver.find_element(By.XPATH, '/html/body/main/section[1]/nav/ul/li[9]/a')
                    next_page.click()
                except Exception as e:
                    print(f"An error occurred: {e}")
                    break


    except Exception as e:
        print(f"An error occurred: {e}")
        break


max_len = max(len(names_list), len(links_list))
names_list += [''] * (max_len - len(names_list))
links_list += [''] * (max_len - len(links_list))

data = {
    'Name': names_list,
    'Link': links_list
}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Close the webdriver
driver.quit()

KeyboardInterrupt: 

In [7]:
df.tail

<bound method NDFrame.tail of                         Name  \
0   WebfortisThe CRM Experts   
1   WebfortisThe CRM Experts   
2   WebfortisThe CRM Experts   
3   WebfortisThe CRM Experts   
4   WebfortisThe CRM Experts   
5   WebfortisThe CRM Experts   
6   WebfortisThe CRM Experts   
7   WebfortisThe CRM Experts   
8   WebfortisThe CRM Experts   
9   WebfortisThe CRM Experts   
10  WebfortisThe CRM Experts   
11  WebfortisThe CRM Experts   
12  WebfortisThe CRM Experts   
13  WebfortisThe CRM Experts   
14  WebfortisThe CRM Experts   
15  WebfortisThe CRM Experts   
16  WebfortisThe CRM Experts   
17  WebfortisThe CRM Experts   

                                                 Link  
0   https://insightglobal.com/?utm_source=clutch.c...  
1   https://insightglobal.com/?utm_source=clutch.c...  
2   https://insightglobal.com/?utm_source=clutch.c...  
3   https://insightglobal.com/?utm_source=clutch.c...  
4   https://insightglobal.com/?utm_source=clutch.c...  
5   https://insightglobal

In [42]:
lunk = df['Link']
lunk

Series([], Name: Link, dtype: float64)

In [43]:
from bs4 import BeautifulSoup
import requests
import requests.exceptions
import urllib.parse
from collections import deque
import re


for link in lunk:
    user_url = link
    urls = deque([user_url])
    scraped_urls = set()
    emails = set()
    
    count = 0
    try:
        while len(urls):
            count += 1
            if count > 100:
                break
            url = urls.popleft()
            scraped_urls.add(url)
            parts = urllib.parse.urlsplit(url)
            base_url = '{0.scheme}://{0.netloc}'.format(parts)
            path = url[:url.rfind('/') + 1] if '/' in parts.path else url
            print('[{}] Processing {}'.format(count, url))
            try:
                response = requests.get(url)
            except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
                continue
    
            new_emails = set(re.findall(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', response.text, re.I))
            emails.update(new_emails)
    
            soup = BeautifulSoup(response.text, 'html.parser')
    
            for anchor in soup.find_all("a"):
                link = anchor.get('href', '')
                if link.startswith('/'):
                    link = base_url + link
                elif link.startswith('http'):
                    link = link
                if link not in urls and link not in scraped_urls:
                    urls.append(link)
    except KeyboardInterrupt:
        print('[-] Closing')
    
    for mail in emails:
        print(mail)